# Deep Learning Project : unsupervised-representation-learning-by-predicting-image-rotations-on-MNIST

## Importing the necessary libraries

In [10]:
import torch
from torchvision import datasets, transforms

## Loading the data

In [6]:
class MinMaxScaler:
    def __call__(self, sample):
        return sample / 255

transform = transforms.Compose([
    transforms.ToTensor(),  
    MinMaxScaler()     
])

In [13]:
X_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
X_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [20]:
X_train.data.shape, X_test.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [19]:
X_train.targets.shape, X_test.targets.shape

(torch.Size([60000]), torch.Size([10000]))

## Spllitting the data for the semi-supervised experiment

In [21]:
X_train_100, X_train_unsupervised = torch.utils.data.Subset(X_train, range(100)), torch.utils.data.Subset(X_train, range(100, len(X_train)))

In [50]:
X_test_loader = torch.utils.data.DataLoader(X_test, batch_size=100, shuffle=False)

## Basline Model ( CNN Trained on 100 samples of supervised data )

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

BATCH_SIZE = 1
EPOCHS = 20

In [46]:
X_train_100_loader = DataLoader(X_train_100, batch_size=BATCH_SIZE, shuffle=True)

In [47]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) # output: 32x28x28
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 32x14x14
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # output: 64x14x14
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output: 64x7x7
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [48]:
basline_model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(basline_model.parameters(), lr=0.001)

In [49]:
basline_model.train()
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(X_train_100_loader):
        outputs = basline_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

Epoch: 0, Loss: 2.311993360519409
Epoch: 1, Loss: 2.350776433944702
Epoch: 2, Loss: 2.3427696228027344
Epoch: 3, Loss: 2.342832565307617
Epoch: 4, Loss: 2.3009705543518066
Epoch: 5, Loss: 2.226966619491577
Epoch: 6, Loss: 2.3344483375549316
Epoch: 7, Loss: 2.2815256118774414
Epoch: 8, Loss: 2.2269797325134277
Epoch: 9, Loss: 2.281541585922241
Epoch: 10, Loss: 2.3009674549102783
Epoch: 11, Loss: 2.276491165161133
Epoch: 12, Loss: 2.2764711380004883
Epoch: 13, Loss: 2.3507964611053467
Epoch: 14, Loss: 2.334423542022705
Epoch: 15, Loss: 2.300990581512451
Epoch: 16, Loss: 2.311993360519409
Epoch: 17, Loss: 2.378126859664917
Epoch: 18, Loss: 2.3009629249572754
Epoch: 19, Loss: 2.300961494445801


### Evaluate the baseline on the test

In [52]:
basline_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in X_test_loader:
        outputs = basline_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total} %')

Accuracy: 9.58 %


## Unsupervised Representation Learning Model